In [2]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt

In [3]:
movie = pd.read_csv("movie.csv")
# "rating.csv" dosyasını okuyarak bir veri çerçevesi oluşturur.
# Ardından "timestamp" sütununu veriden çıkarır.
rating = pd.read_csv("rating.csv").drop("timestamp",axis=1)

In [4]:
print(movie.dtypes)
print(rating.dtypes)

movieId     int64
title      object
genres     object
dtype: object
userId       int64
movieId      int64
rating     float64
dtype: object


In [5]:
movie["movieId"] = movie["movieId"].astype(object)
rating["movieId"] = rating["movieId"].astype(object)
rating["userId"] = rating["userId"].astype(object)

In [6]:
print(movie.isnull().sum())
print(rating.isnull().sum())

movieId    0
title      0
genres     0
dtype: int64
userId     0
movieId    0
rating     0
dtype: int64


In [7]:
C = rating["rating"].mean()
print(C)

3.5255285642993797


In [8]:
# Veri çerçevesindeki "movieId" sütununa göre gruplama yapılır ve her bir film için oy sayısını hesaplar.
rating_count = rating.groupby(["movieId"]).count()

# Yeni bir "movieId" sütunu oluşturarak gruplama sonuçlarını bu sütunda saklar.
rating_count["movieId"] = rating_count.index

# İndeks adını "None" olarak değiştirir.
rating_count.index.name = None

# İndeksi sıfırlayarak, yeni "movieId" sütununu eski indeks olarak kullanır.
rating_count.reset_index(inplace=True)

# Gereksiz "index" sütununu kaldırır.
del rating_count["index"]

rating_count



,userId,rating,movieId
0,49695,49695,1
1,22243,22243,2
2,12735,12735,3
3,2756,2756,4
4,12161,12161,5
...,...,...,...
26739,1,1,131254
26740,1,1,131256
26741,1,1,131258
26742,1,1,131260


In [9]:
means = rating.groupby(["movieId"]).mean()
means["movieId"] = means.index
means.index.name = None
means.reset_index(inplace=True)
del means["index"]
means = means.rename(columns={"rating":"mean"})

means

,userId,mean,movieId
0,69282.396821,3.921240,1
1,69169.928202,3.211977,2
2,69072.079388,3.151040,3
3,69652.91328,2.861393,4
4,69113.475454,3.064592,5
...,...,...,...
26739,79570.0,4.000000,131254
26740,79570.0,4.000000,131256
26741,28906.0,2.500000,131258
26742,65409.0,3.000000,131260


In [10]:
df = pd.merge(movie, rating_count)
df = pd.merge(df, means)

print(df.head)

<bound method NDFrame.head of Empty DataFrame
Columns: [movieId, title, genres, userId, rating, mean]
Index: []>


## persentilde ( yüzdelik dilim )

In [11]:
# Veri çerçevesindeki "userId" sütunundaki değerlerin %90'ını belirleyerek eşik değeri oluşturur.
m = df["userId"].quantile(0.90)

# Orijinal veri çerçevesini kopyalar ve sadece "userId" sütunundaki değeri eşik değerden büyük veya eşit olanları içerecek şekilde filtreler.
q_movies = df.copy().loc[df["userId"] >= m]

q_movies.shape


(0, 6)

In [ ]:
# Bir film için ağırlıklı bir puan hesaplamak üzere kullanılacak fonksiyonu tanımlar.
# Bu fonksiyon, kullanıcı sayısı (v), ortalama puan (R), eşik değeri (m) ve normalleştirme sabiti (C) parametrelerini alır.
def weight_rating(x, m=m, C=C):
    v = x["userId"]  # Kullanıcı sayısını alır.
    R = x["mean"]    # Ortalama puanı alır.
    
    # Ağırlıklı puan formülüne göre ağırlıklı puanı hesaplar ve döndürür.
    return (v / (v + m) * R) + (m / (m + v) * C)

# "q_movies" veri çerçevesine "weighted_rating" adında yeni bir sütun ekler.
# Bu sütun, "weight_rating" fonksiyonunu kullanarak her bir film için ağırlıklı puanı içerir.
q_movies["weighted_rating"] = q_movies.apply(weight_rating, axis=1)

# "q_movies" veri çerçevesini "score" sütununa göre büyükten küçüğe sıralar.
q_movies = q_movies.sort_values('score', ascending=False)


q_movies.head(10)


* kalp pileri yapılan saldırı arttı önleme